In [1]:
import warnings
import numpy as np
import matplotlib.pyplot as plt
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
torch.manual_seed(0)

import networkx as nx
import pandas as pd

from sklearn.neighbors import KernelDensity

from ConDist import CsvDataset, ConDist_MO, Condist_for_bn
from reinforcement import environment, Policy, reinforce, evaluate
from metrics import causal_edge_score, causal_cfe_from_parent, sparsity_num, sparsity_cat, L1, cess_for_bn
warnings.filterwarnings("ignore")

In [2]:
import pickle
with open('./bn_data/scm.pickle', 'rb') as f:
    SCM = pickle.load(f)

def func_gen(w):
    def func(*args):
        return np.array(w).dot(np.array([1]+list(args)))
    return func

for k in SCM.keys():
    SCM[k]['func']  = func_gen(w=SCM[k]['weight'])

In [3]:
device = 'cpu'

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.compose import ColumnTransformer


In [5]:
clf_data_train = pd.read_csv('./bn_data/sangiovese_data_train.csv')
clf_data_test = pd.read_csv('./bn_data/sangiovese_data_test.csv')

In [6]:
target = clf_data_train["y"]
# Split data into train and test
x_train= clf_data_train.drop("y", axis=1)

categorical = []
numerical = x_train.columns.difference(categorical)

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='error'))])

transformations = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical),
        ('cat', categorical_transformer, categorical)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', transformations),
                      ('classifier', MLPClassifier(random_state=0))])
model = clf.fit(x_train, clf_data_train['y'])

In [7]:
np.mean(clf.predict(clf_data_test.drop('y', axis=1)) == clf_data_test['y'])

0.8236819360414867

In [8]:
pred_train = clf.predict(clf_data_train.drop('y', axis=1)).astype('int')
pred_test = clf.predict(clf_data_test.drop('y', axis=1)).astype('int')
cf_data_train = clf_data_train.drop('y', axis=1)[pred_train==0]
cf_data_test = clf_data_test.drop('y', axis=1)[pred_test==0]
scm = SCM

In [9]:
# For this dataset, the conditional distribution is given by the bayesian network.
# The conditional distribution should be estimated at normalized scale
cat_feats = []
train_num_mean = clf_data_train.drop('y', axis=1).mean()
train_num_std = clf_data_train.drop('y', axis=1).std()
train_data_norm = clf_data_train.drop('y', axis=1)
train_data_norm = (train_data_norm - train_num_mean)/ train_num_std
dist_nets = {}
for out_feat in scm.keys():
    in_feats = scm[out_feat]['input']
    if in_feats:
        dist_nets[out_feat] = {}
        dist_nets[out_feat]['input'] = in_feats
        dist_nets[out_feat]['net'] = Condist_for_bn(out_feat, scm, train_num_mean, train_num_std)

In [10]:
DONE_THRESHOLD = 0.5
REG_BETA = 0.7
CON_DIST_BETA = 0.7
ACTION_ENTROPY_BETA = 0.0001
N_ALLOWED_STD = 0.2
MAX_STEP = 13
CAT_CAHNGE_PENALTY = 0.3
ADD_BIAS = False
cat_feats = []
cat_feats_names = []
n_classes = []
disallowed_feats = []
directions = {}
policy = Policy(n_feats=13, cat_feats=cat_feats, n_classes=n_classes, disallowed_feats=disallowed_feats, hidden_size=128).to(device)
optimizer = optim.Adam(policy.parameters(), lr=1e-4)
env = environment(clf, clf_data_train.drop('y',axis=1).mean().to_numpy(), clf_data_train.drop('y',axis=1).std().to_numpy(),\
     train_data_norm.min().to_numpy(), train_data_norm.max().to_numpy(), cat_feats, list(cf_data_train.columns), \
     scm, DONE_THRESHOLD, REG_BETA, cat_change_penalty=CAT_CAHNGE_PENALTY, max_step=MAX_STEP, disallowd_feats=disallowed_feats, directions=directions, order_matter=False, add_bias_to_endo=ADD_BIAS)
# scores = reinforce(cf_data_train, env, policy, optimizer, checkpoint_path='./ckpt_bn', n_episodes=30000, print_every=1000, con_dist=dist_nets, n_allowed_std=N_ALLOWED_STD, CON_DIST_BETA=CON_DIST_BETA, ACTION_ENTROPY_BETA = ACTION_ENTROPY_BETA, device=device)

In [11]:
with open('./ckpt_bn/model_12000.pt', 'rb') as f:
    policy = torch.load(f)

In [12]:
r = evaluate(cf_data_test, env, policy, device=device, print_file='./result.txt')
cfs = pd.DataFrame(r['CF'], columns=cf_data_test.columns)

In [13]:
mask = clf.predict(cfs)
mask = [i for i in range(len(mask)) if mask[i]]

In [14]:
cess_cf = cess_for_bn(cf_data_test.iloc[mask].to_numpy(), cfs.iloc[mask].to_numpy(), SCM)

In [24]:
print('validity:', np.mean(model.predict(cfs)))
print('causal_edge_score(median): ', np.mean(np.median(cess_cf, axis=1)))
print('causal_edge_score(average): ', np.mean(cess_cf))
print('sparsity_num: ', sparsity_num(cfs.iloc[mask], cf_data_test.iloc[mask], cat_feats=[]))
print('L1: ', L1(cfs.iloc[mask], cf_data_test.iloc[mask], cat_feats=[], stds=clf_data_train.std()))

validity: 1.0
causal_edge_score(median):  0.2000142843394448
causal_edge_score(average):  0.2998812484146612
sparsity_num:  0.1332871332871333
L1:  1.1318849676888327


In [25]:
causal_cfs_from_parent = causal_cfe_from_parent(cfs, cf_data_test, scm=SCM, cat_feats=[], add_bias=False)
exo_node = [feat for feat in SCM.keys() if SCM[feat]['input'] == []]
exo_feats = cf_data_test.columns[exo_node]
# Modify the exogenous back to the original for computing L1/ sparsity from causal perspective 
causal_cfs_from_parent[exo_feats] = cf_data_test[exo_feats].to_numpy()
print('sparsity_num(causal perspective): ', sparsity_num(cfs.iloc[mask], causal_cfs_from_parent.iloc[mask], cat_feats=[]))
print('L1(causal perspective): ', L1(cfs.iloc[mask], causal_cfs_from_parent.iloc[mask], cat_feats=[], stds=clf_data_train.std()))

sparsity_num(causal perspective):  0.8044583044583045
L1(causal perspective):  0.26513492693685703
